# 1. Loading extracted data

In [106]:
# Import packages
import pandas as pd
pd.set_option('display.max_columns', None)

In [107]:
df = pd.read_csv('aviation_incidents.csv', encoding = 'latin1')
df.head()

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Total airframe hrs,Cycles,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description
0,Saturday 20 March 2021,11:26,Boeing 737-4Y0,Trigana Air Service,PK-YSF,23869/1639,1988.0,55982 hours,65005 flights,CFMI CFM56-3C1,Fatalities: 0 / Occupants: 4,0,"Substantial, written off",Accident,Jakarta-Halim Perdana Kusuma Airport (HLP) - I...,Landing,Cargo,Jakarta-Halim Perdana Kusuma Airport (HLP/WIHH),Makassar-Sultan Hasanuddin International Airpo...,NTSC,Information verified through data from acciden...,A Boeing 737-4Y0 of Trigana Air Service suffer...
1,Tuesday 23 March 2021,NaN,Saab 340AF,SprintAir,SP-KPU,340A-145,1989.0,NaN,NaN,General Electric CT7-5A2,Fatalities: 0 / Occupants: 0,0,"Substantial, written off",Accident,Gdansk-Lech Walesa Airport (GDN) - Poland,Standing,-,-,-,SCAAI,Information verified through data from acciden...,"n 23 March 2021 in the morning, the pilot of t..."
2,Sunday 4 July 2021,07:00,Antonov An-2R,ELISA,UP-A0135,1G143-48,1973.0,NaN,NaN,Shvetsov ASh-62IR,Fatalities: 0 / Occupants: 2,0,Substantial,Accident,"12 km from Karaultobe, Kyzylorda - Kazakhstan","Manoeuvring (airshow, firefighting, ag.ops.)",Agricultural,NaN,NaN,NaN,"Information is only available from news, socia...",An Antonov An-2 force landed in a field after ...
3,Friday 26 March 2021,16:30,Beechcraft B200 Super King Air,West Wind Aviation Ltd.,5Y-NJS,BB-837,1981.0,12177 hours,NaN,Pratt & Whitney Canada PT6A-42,Fatalities: 0 / Occupants: 3,0,"Destroyed, written off",Accident,Nairobi - Kenya,Approach,Private,Eldoret Airport (EDL/HKEL),Nairobi-Wilson Airport (WIL/HKNW),NaN,Information verified through data from acciden...,The aircraft made a forced landing on the Ngon...
4,Tuesday 5 October 2021,05:44,Dassault Falcon 20C,Sierra West Airlines,N283SA,83,1967.0,18798 hours,NaN,NaN,Fatalities: 2 / Occupants: 2,0,"Destroyed, written off",Accident,"1,3 km W of Thomson-McDuffie County Airport, G...",Approach,Cargo,"Lubbock Preston Smith International Airport, T...","Thomson-McDuffie County Airport, GA (KHQU)",NTSB,Accident investigation report completed and in...,"Sierra West Airlines flight PKW887, a Dassault..."


In [108]:
# Filtering only passenger flights
df.dropna(subset = 'Nature', inplace = True)
passenger = df[df['Nature'].str.contains("Passenger")]
passenger.head()

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Total airframe hrs,Cycles,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description
15,Friday 16 July 2021,16:11,Antonov An-28,SiLA - Siberian Light Aviation,RA-28728,1AJ007-13,1989.0,8698 hours,5921 flights,Glushenkov TVD-10B,Fatalities: 0 / Occupants: 19,0,Substantial,Accident,52 km SE of Kedrovo - Russia,En route,Passenger - Scheduled,Kedrovo Airport,Tomsk-Bogashevo Airport (TOF/UNTT),MAK,Accident investigation report completed and in...,An Antonov An-28 airplane flipped over during ...
20,Monday 8 February 2021,11:02,Embraer EMB-500 Phenom 100,Luxwing,9H-FAM,50000100,2011.0,NaN,NaN,Pratt & Whitney Canada PW615F,Fatalities: 0 / Occupants: 3,0,"Substantial, written off",Accident,Paris-Le Bourget Airport (LBG) - France,Landing,Passenger - Non-Scheduled/charter/Air Taxi,Venice-Marco Polo Airport (VCE/LIPZ),Paris-Le Bourget Airport (LBG/LFPB),BEA,Accident investigation report completed and in...,Shortly before landing on runway 27 at Paris-L...
21,Friday 3 September 2021,12:44,Raytheon 390 Premier I,PB Air LLC,N351CW,RB-53,2002.0,3105 hours,NaN,Williams International FJ44-2A,Fatalities: 0 / Occupants: 4,0,Substantial,Accident,"Naples Airport, FL (APF) - United States of Am...",Landing,Passenger - Non-Scheduled/charter/Air Taxi,"Indianapolis International Airport, IN (IND/KIND)","Naples Airport, FL (APF/KAPF)",NTSB,Accident investigation report completed and in...,"A Raytheon 390, N351CW, was substantially dama..."
23,Saturday 23 October 2021,10:55 LT,Embraer ERJ-175LR,"Envoy Air, opf American Eagle",N252NN,17000642,2017.0,NaN,NaN,GE CF34-8E5,Fatalities: 0 / Occupants: 63,0,None,Accident,"Miami International Airport, FL (MIA/KMIA) - U...",Pushback / towing,Passenger - Scheduled,"Miami International Airport, FL (MIA/KMIA)","Raleigh/Durham Airport, NC (RDU/KRDU)",NTSB,Accident investigation report completed and in...,Envoy flight 4265 experienced a jolt during pu...
24,Friday 12 March 2021,16:46,Cessna 208B Grand Caravan,Grant Aviation,N407GV,208B0616,1997.0,27418 hours,NaN,Pratt & Whitney Canada PT6A-114A,Fatalities: 0 / Occupants: 5,0,"Substantial, repaired",Accident,"Chevak Airport, AK (VAK) - United States of Am...",Take off,Passenger - Scheduled,"Chevak Airport, AK (VAK/PAVA)","Bethel Airport, AK (BET/PABE)",NTSB,Accident investigation report completed and in...,Aircraft aborted takeoff due to crosswind and ...


In [109]:
passenger.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429 entries, 15 to 878
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  427 non-null    object 
 1   Time                  372 non-null    object 
 2   Type                  429 non-null    object 
 3   Owner/operator        428 non-null    object 
 4   Registration          427 non-null    object 
 5   MSN                   421 non-null    object 
 6   Year of manufacture   378 non-null    float64
 7   Total airframe hrs    105 non-null    object 
 8   Cycles                31 non-null     object 
 9   Engine model          348 non-null    object 
 10  Fatalities            429 non-null    object 
 11  Other fatalities      429 non-null    int64  
 12  Aircraft damage       429 non-null    object 
 13  Category              425 non-null    object 
 14  Location              429 non-null    object 
 15  Phase                 

In [110]:
# Dropping cycles and total airframe hrs as there are a number of null values
passenger.drop(columns = ['Total airframe hrs','Cycles'], inplace = True)
passenger = passenger.dropna(subset = ['Owner/operator'])
passenger.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 15 to 878
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  426 non-null    object 
 1   Time                  371 non-null    object 
 2   Type                  428 non-null    object 
 3   Owner/operator        428 non-null    object 
 4   Registration          427 non-null    object 
 5   MSN                   421 non-null    object 
 6   Year of manufacture   378 non-null    float64
 7   Engine model          348 non-null    object 
 8   Fatalities            428 non-null    object 
 9   Other fatalities      428 non-null    int64  
 10  Aircraft damage       428 non-null    object 
 11  Category              424 non-null    object 
 12  Location              428 non-null    object 
 13  Phase                 428 non-null    object 
 14  Nature                428 non-null    object 
 15  Departure airport     

/tmp/ipykernel_3328/1945045275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passenger.drop(columns = ['Total airframe hrs','Cycles'], inplace = True)


# 2.Column alterations

In [111]:
from datetime import datetime

## DateTime

In [112]:
# Using UTC time zone for now may consider using local time zone later

pattern = r'(\d{2}:\d{2})'

dtf = passenger['Date'] + ' ' + passenger['Time'].str.extract(pattern, expand = False).fillna('00:00')

passenger['DateTime'] = pd.to_datetime(dtf)
passenger.head()

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description,DateTime
15,Friday 16 July 2021,16:11,Antonov An-28,SiLA - Siberian Light Aviation,RA-28728,1AJ007-13,1989.0,Glushenkov TVD-10B,Fatalities: 0 / Occupants: 19,0,Substantial,Accident,52 km SE of Kedrovo - Russia,En route,Passenger - Scheduled,Kedrovo Airport,Tomsk-Bogashevo Airport (TOF/UNTT),MAK,Accident investigation report completed and in...,An Antonov An-28 airplane flipped over during ...,2021-07-16 16:11:00
20,Monday 8 February 2021,11:02,Embraer EMB-500 Phenom 100,Luxwing,9H-FAM,50000100,2011.0,Pratt & Whitney Canada PW615F,Fatalities: 0 / Occupants: 3,0,"Substantial, written off",Accident,Paris-Le Bourget Airport (LBG) - France,Landing,Passenger - Non-Scheduled/charter/Air Taxi,Venice-Marco Polo Airport (VCE/LIPZ),Paris-Le Bourget Airport (LBG/LFPB),BEA,Accident investigation report completed and in...,Shortly before landing on runway 27 at Paris-L...,2021-02-08 11:02:00
21,Friday 3 September 2021,12:44,Raytheon 390 Premier I,PB Air LLC,N351CW,RB-53,2002.0,Williams International FJ44-2A,Fatalities: 0 / Occupants: 4,0,Substantial,Accident,"Naples Airport, FL (APF) - United States of Am...",Landing,Passenger - Non-Scheduled/charter/Air Taxi,"Indianapolis International Airport, IN (IND/KIND)","Naples Airport, FL (APF/KAPF)",NTSB,Accident investigation report completed and in...,"A Raytheon 390, N351CW, was substantially dama...",2021-09-03 12:44:00
23,Saturday 23 October 2021,10:55 LT,Embraer ERJ-175LR,"Envoy Air, opf American Eagle",N252NN,17000642,2017.0,GE CF34-8E5,Fatalities: 0 / Occupants: 63,0,None,Accident,"Miami International Airport, FL (MIA/KMIA) - U...",Pushback / towing,Passenger - Scheduled,"Miami International Airport, FL (MIA/KMIA)","Raleigh/Durham Airport, NC (RDU/KRDU)",NTSB,Accident investigation report completed and in...,Envoy flight 4265 experienced a jolt during pu...,2021-10-23 10:55:00
24,Friday 12 March 2021,16:46,Cessna 208B Grand Caravan,Grant Aviation,N407GV,208B0616,1997.0,Pratt & Whitney Canada PT6A-114A,Fatalities: 0 / Occupants: 5,0,"Substantial, repaired",Accident,"Chevak Airport, AK (VAK) - United States of Am...",Take off,Passenger - Scheduled,"Chevak Airport, AK (VAK/PAVA)","Bethel Airport, AK (BET/PABE)",NTSB,Accident investigation report completed and in...,Aircraft aborted takeoff due to crosswind and ...,2021-03-12 16:46:00


In [113]:
passenger.loc[passenger['DateTime'].isnull()]

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description,DateTime
453,NaN,NaN,Airbus A330-941,Delta Air Lines,N411DX,1991,NaN,NaN,Fatalities: 0 / Occupants:,0,Unknown,NaN,Paris-Charles de Gaulle Airport (CDG/LFPG) - F...,Taxi,Passenger - Scheduled,"Seattle/Tacoma International Airport, WA (SEA/...",Paris-Charles de Gaulle Airport (CDG/LFPG),NaN,"Information is only available from news, socia...","Air France flight AF483, a Boeing 777-328ER (F...",NaT
532,NaN,NaN,Boeing 777-328ER,Air France,F-GSQT,32846/616,NaN,NaN,Fatalities: 0 / Occupants:,0,Unknown,NaN,Paris-Charles de Gaulle Airport (CDG/LFPG) - F...,Taxi,Passenger - Scheduled,Lima-Jorge ChÃ¡vez International Airport (LIM/...,Paris-Charles de Gaulle Airport (CDG/LFPG),NaN,"Information is only available from news, socia...","Air France flight AF483, a Boeing 777-328ER (F...",NaT


In [114]:
# Reassign date and time
passenger.loc[passenger['DateTime'].isnull(),'DateTime'] = pd.to_datetime('2023-08-30 13:31')

In [115]:
passenger.DateTime.isnull().sum()

0

## Operator

In [116]:
# Lists to drop, including chartered flights, after Chat GPT result, and double checking with iata web page
non_commercial_operators = [
    "2nd Arkhangelsk United Aviation Division",
    "987 Investments LLC",
    "AB Aviation",
    "ART Táxi Aéreo Ltda",
    "Abyssinian Flight Services, opf UNWorld Food Programme",
    "Ace Aviation Services dba Hop-A-Jet",
    "Aeroservice, opf SiLA - Siberian Light Aviation",
    "Air Taxi Vanuatu",
    "Airfast, lsf Air Serv",
    "Alkan Air",
    "Anderson Air",
    "Angkasa Super Services",
    "Antarctic Airways",
    "BAR Aviation",
    "Bald Mountain Air Services",
    "Brooks Range Aviation",
    "Care Aviation",
    "Caverton Helicopters (Cameroun) Ltd.",
    "Cetraca Aviation Service",
    "Chartright Air",
    "Corporate Flight Management Inc dba Contour Aviation",
    "ESAV Airlines",
    "Eagle Jet 300 LLC",
    "Equaflight Services",
    "Flamingo Air Charter",
    "Flexjet LLC",
    "Flints Aero Services",
    "Flytec SA",
    "Frayer Jets 7 LLC, opb GC Aviation",
    "Freedom Airline Express",
    "Friday Harbor Seaplane Tours",
    "Gomair",
    "Grant Aviation",
    "Halsted Aviation Corporation (HAC)",
    "Helifix - PNG",
    "International Jet Management",
    "Jags Aviation",
    "Jet It LLC",
    "Jet Valet Sdn Bhd",
    "Jupiter Aviation LLC",
    "Kamchatka Aviation Enterprise",
    "Kobio Aviation",
    "LeAir Charter Services",
    "Light Air Services, lsf Eagle Air",
    "Lowcountry Aviation Co Brokerage LLC",
    "Luxwing",
    "Manaus Aerotaxi",
    "Mobil Avia",
    "NDK Air Limited",
    "Naryan-Mar Air Enterprise",
    "NetJets",
    "Nolinor Aviation",
    "North Coast Aviation",
    "Northwestern Air",
    "Oriental Air Bridge",
    "Ozark Air Services",
    "PB Air LLC",
    "Penjet Pty Ltd",
    "Perimeter Aviation",
    "Piquiatuba Táxi Aéreo",
    "RICO Taxi Aéreo",
    "Raber Flight Services, LLC",
    "Renegade Air, opf UN Humanitarian Air Service",
    "Royal Skyline Ltd",
    "Rust's Flying Service",
    "SARAT Flight Logics",
    "SVG Air",
    "Secretaria da Segurança Pública e Defesa Social Pará",
    "SiLA - Siberian Light Aviation",
    "Sindbard Air",
    "SkyJet",
    "Skystallion LLC",
    "Skyward Express",
    "Smart Cakrawala Aviation",
    "Swan River Seaplanes",
    "Sydney Seaplanes",
    "TAG Airlines",
    "Tara Air",
    "Tarco Aviation, lsf Mid Africa Aviation",
    "Thai Flying Service Co Ltd",
    "Thunder Airlines",
    "Torres Strait Air",
    "Transition Aviation",
    "Ultra Air, LLC",
    "UnityAir Zanzibar",
    "Unknown",
    "VSR Ventures",
    "VistaJet",
    "Westair Aviation",
    "Willis Lease Finance Corporation",
    "Windward Express Airways",
    "Wright Air Service",
    "Z.Boskovic Air Charters",
    "private"
]

In [117]:
# Now the list only includes commercial passenger airlines
passenger = passenger[~passenger['Owner/operator'].isin(non_commercial_operators)]
passenger.head()

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description,DateTime
23,Saturday 23 October 2021,10:55 LT,Embraer ERJ-175LR,"Envoy Air, opf American Eagle",N252NN,17000642,2017.0,GE CF34-8E5,Fatalities: 0 / Occupants: 63,0,None,Accident,"Miami International Airport, FL (MIA/KMIA) - U...",Pushback / towing,Passenger - Scheduled,"Miami International Airport, FL (MIA/KMIA)","Raleigh/Durham Airport, NC (RDU/KRDU)",NTSB,Accident investigation report completed and in...,Envoy flight 4265 experienced a jolt during pu...,2021-10-23 10:55:00
26,Sunday 13 June 2021,23:21,Boeing 737-8AS (WL),Ryanair,EI-EVE,35035/3920,2012.0,CFMI CFM56-7B,Fatalities: 0 / Occupants: 155,0,None,Accident,MÃ¡laga-Costa del Sol Airport (AGP/LEMG) - Spain,Standing,Passenger - Scheduled,Bruxelles-Sud Charleroi Airport (CRL/EBCI),MÃ¡laga-Costa del Sol Airport (AGP/LEMG),CIAIAC,Accident investigation report completed and in...,"The Boeing 737-8AS aircraft, with registration...",2021-06-13 23:21:00
27,Monday 15 March 2021,15:00 LT,Boeing 777-223ER,American Airlines,N789AN,30252/285,2000.0,Rolls-Royce RB.211 SERIES,Fatalities: 0 / Occupants: 262,0,None,Accident,"Fort Worth, Texas - United States of America",En route,Passenger - Scheduled,CancÃºn Airport (CUN/MMUN),"Fort Worth, TX",NTSB,Accident investigation report completed and in...,"On March 15, 2021, about 1500 central standard...",2021-03-15 15:00:00
29,Friday 21 May 2021,12:30 LT,Boeing 737-7H4,Southwest Airlines,N289CT,36938/5892,NaN,CFMI CFM56-7B,Fatalities: 0 / Occupants: 107,0,Minor,Accident,"Chicago-O'Hare International Airport, IL (ORD/...",Taxi,Passenger - Scheduled,"Washington-Dulles International Airport, DC (I...","Chicago-O'Hare International Airport, IL (ORD/...",NTSB,Accident investigation report completed and in...,"On May 21, 2021, at about 1230 central dayligh...",2021-05-21 12:30:00
37,Friday 28 May 2021,12:40,Boeing 737-823 (WL),American Airlines,N834NN,29576/3244,2010.0,CFMI CFM56-7B24/3,Fatalities: 0 / Occupants: 166,0,"Substantial, repaired",Accident,"Dallas/Fort Worth International Airport, TX (D...",Taxi,Passenger - Scheduled,"Dallas/Fort Worth International Airport, TX (D...",Nassau-Lynden Pindling International Airport (...,NTSB,Accident investigation report completed and in...,"American Airlines flight 1005, a Boeing 737-80...",2021-05-28 12:40:00


In [118]:
passenger.iloc[0,:]

Date                                             Saturday 23 October 2021
Time                                                             10:55 LT
Type                                                    Embraer ERJ-175LR
Owner/operator                              Envoy Air, opf American Eagle
Registration                                                       N252NN
MSN                                                              17000642
Year of manufacture                                                2017.0
Engine model                                                  GE CF34-8E5
Fatalities                                  Fatalities: 0 / Occupants: 63
Other fatalities                                                        0
Aircraft damage                                                      None
Category                                                         Accident
Location                Miami International Airport, FL (MIA/KMIA) - U...
Phase                                 

In [119]:
# Cleaning operator

In [ ]:
operators = pd.DataFrame(passenger['Owner/operator'])

for ind, row in operators.iterrows():
    if ' opf ' in row[0]:
        row['operating_carrier'], row['marketing_carrier'] = row[0].replace(',','').split(' opf ')
    elif ' opb ' in row[0]:
        row['marketing_carrier'], row['operating_carrier'] = row[0].replace(',','').split(' opb ')
    else:
        row['operating_carrier'] = row[0]
        row['marketing_carrier'] = row[0]
    passenger.loc[ind,'operating_carrier'] = row['operating_carrier']
    passenger.loc[ind,'marketing_carrier'] = row['marketing_carrier']
passenger

,Date,Time,Type,Owner/operator,Registration,MSN,Year of manufacture,Engine model,Fatalities,Other fatalities,Aircraft damage,Category,Location,Phase,Nature,Departure airport,Destination airport,Investigating agency,Confidence Rating,Description,DateTime,operating_carrier,marketing_carrier
23,Saturday 23 October 2021,10:55 LT,Embraer ERJ-175LR,"Envoy Air, opf American Eagle",N252NN,17000642,2017.0,GE CF34-8E5,Fatalities: 0 / Occupants: 63,0,None,Accident,"Miami International Airport, FL (MIA/KMIA) - U...",Pushback / towing,Passenger - Scheduled,"Miami International Airport, FL (MIA/KMIA)","Raleigh/Durham Airport, NC (RDU/KRDU)",NTSB,Accident investigation report completed and in...,Envoy flight 4265 experienced a jolt during pu...,2021-10-23 10:55:00,Envoy Air,American Eagle
26,Sunday 13 June 2021,23:21,Boeing 737-8AS (WL),Ryanair,EI-EVE,35035/3920,2012.0,CFMI CFM56-7B,Fatalities: 0 / Occupants: 155,0,None,Accident,MÃ¡laga-Costa del Sol Airport (AGP/LEMG) - Spain,Standing,Passenger - Scheduled,Bruxelles-Sud Charleroi Airport (CRL/EBCI),MÃ¡laga-Costa del Sol Airport (AGP/LEMG),CIAIAC,Accident investigation report completed and in...,"The Boeing 737-8AS aircraft, with registration...",2021-06-13 23:21:00,Ryanair,Ryanair
27,Monday 15 March 2021,15:00 LT,Boeing 777-223ER,American Airlines,N789AN,30252/285,2000.0,Rolls-Royce RB.211 SERIES,Fatalities: 0 / Occupants: 262,0,None,Accident,"Fort Worth, Texas - United States of America",En route,Passenger - Scheduled,CancÃºn Airport (CUN/MMUN),"Fort Worth, TX",NTSB,Accident investigation report completed and in...,"On March 15, 2021, about 1500 central standard...",2021-03-15 15:00:00,American Airlines,American Airlines
29,Friday 21 May 2021,12:30 LT,Boeing 737-7H4,Southwest Airlines,N289CT,36938/5892,NaN,CFMI CFM56-7B,Fatalities: 0 / Occupants: 107,0,Minor,Accident,"Chicago-O'Hare International Airport, IL (ORD/...",Taxi,Passenger - Scheduled,"Washington-Dulles International Airport, DC (I...","Chicago-O'Hare International Airport, IL (ORD/...",NTSB,Accident investigation report completed and in...,"On May 21, 2021, at about 1230 central dayligh...",2021-05-21 12:30:00,Southwest Airlines,Southwest Airlines
37,Friday 28 May 2021,12:40,Boeing 737-823 (WL),American Airlines,N834NN,29576/3244,2010.0,CFMI CFM56-7B24/3,Fatalities: 0 / Occupants: 166,0,"Substantial, repaired",Accident,"Dallas/Fort Worth International Airport, TX (D...",Taxi,Passenger - Scheduled,"Dallas/Fort Worth International Airport, TX (D...",Nassau-Lynden Pindling International Airport (...,NTSB,Accident investigation report completed and in...,"American Airlines flight 1005, a Boeing 737-80...",2021-05-28 12:40:00,American Airlines,American Airlines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,Wednesday 29 January 2025,20:48 LT,Bombardier CRJ-701ER (CL-600-2C10),"American Eagle, opb PSA Airlines",N709PS,10165,2004.0,GE CF34-8C5,Fatalities: 64 / Occupants: 64,3,"Destroyed, written off",Accident,near Washington-Ronald Reagan National Airport...,Approach,Passenger - Scheduled,"Wichita Dwight D. Eisenhower National Airport,...","Washington-Ronald Reagan National Airport, DC ...",NTSB,"Information is only available from news, socia...","American Eagle flight 5342, a Bombardier CRJ-7...",2025-01-29 20:48:00,PSA Airlines,American Eagle
874,Wednesday 5 February 2025,c. 17:55 UTC,Boeing 737-832 (WL),Delta Air Lines,N3737C,30799/712,2000.0,CFMI CFM56-7B,Fatalities: 0 / Occupants:,0,Substantial,Accident,"Seattle/Tacoma International Airport, WA (SEA/...",Standing,Passenger - Scheduled,"Seattle/Tacoma International Airport, WA (SEA/...",Puerto Vallarta-Gustavo D. Ordaz Airport (PVR/...,NTSB,"Information is only available from news, socia...","Japan Airlines flight JL68, a Boeing 787-9 Dre...",2025-02-05 17:55:00,Delta Air Lines,Delta Air Lines
875,Saturday 1 February 2025,NaN,Bombardier CRJ-200LR,"American Eagle, opb Air Wisconsin",N420AW,7640,2002.0,GE CF34-3B1,Fatalities: 0 / O

In [151]:
passenger.Category.value_counts()

Accident                 317
Unlawful Interference      4
Serious incident           4
Other                      4
Incident                   1
Name: Category, dtype: int64

In [ ]:
from pydantic import BaseModel
from enum import Enum
from openai import OpenAI
from keys import *

client = OpenAI(api_key = OPENAI_KEY)

# Structured output based on the text

class EventLabel(str, Enum):
    CRASH = 'crash'
    ACCIDENT = 'accident'
    INCIDENT = 'incident'
    def __str__(self):
        return self.value

class CauseLabel(str, Enum):
    PILOT = 'pilot'
    CREW = 'crew'
    TECHNICAL_FAILURE = 'technical failure'
    WEATHER = 'weather condition'
    MISCOMMUNICATION = 'miscommunication'
    BIRD_STRIKE = 'bird strike'
    OTHER = 'other'
    def __str__(self):
        return self.value

class InvestigationLabel(str, Enum):
    ONGOING = 'on-going'
    COMPLETED = 'completed'
    NOT_AVAILABLE = 'not available'
    def __str__(self):
        return self.value

class IncidentDetails(BaseModel):
    #Datetime : datetime
    type_of_event : EventLabel
    operating_carrier : str
    marketing_carrier : str
    Registration : str
    MSN_Manufacturer_Serial_Number : str
    flight_code : str
    plane_manufacturer : str
    plane_model : str
    year_of_manufacture : str
    from_airport : str
    from_airport_code : str
    from_city :str
    from_country :str
    to_airport : str
    to_airport_code : str
    to_city : str
    to_country : str
    incident_location : str
    onboard_passengers :int
    onboard_crews : int
    fatalities : int
    injuries_critical : int
    injuries_minor : int
    injuries_total : int
    cause : CauseLabel
    cause_probability_pct : int
    cause_detailed : str
    investigating_agency: str
    investigation_status : InvestigationLabel

dfs = []

for idx, row in passenger[:1].iterrows():
    response = client.beta.chat.completions.parse(
    model="gpt-4o-mini-2024-07-18",
    temperature = 0,
    store=True,
    messages=[
        {"role" : "system", "content": "You are analyzing commercial airplane incident document based on the given criteria. In case of flight_code, search the flight code of the carrier between the two destinations. \
        In case of distinguishing incident from accident, use ICAO's definition where inncident : 'An occurrence, other than an accident, associated with the operation of an aircraft, which affects or could affect the safety of operation.', and accident : 'An occurrence associated with the operation of an aircraft in which a person is fatally or seriously injured, or the aircraft sustains damage or structural failure, or the aircraft is missing or completely inaccessible.'"},
        {"role" : 'user', "content" : str(row.to_dict())}
    ],
    response_format = IncidentDetails,
    )
    gpt_output = pd.DataFrame(response.choices[0].message.parsed).set_index(0).T
    dfs.append(gpt_output)

In [156]:
dfs[0]

,type_of_event,operating_carrier,marketing_carrier,Registration,MSN_Manufacturer_Serial_Number,flight_code,plane_manufacturer,plane_model,year_of_manufacture,from_airport,from_airport_code,from_city,from_country,to_airport,to_airport_code,to_city,to_country,incident_location,onboard_passengers,onboard_crews,fatalities,injuries_critical,injuries_minor,injuries_total,cause,cause_probability,cause_detailed,investigating_agency,investigation_status
1,incident,Envoy Air,American Eagle,N252NN,17000642,AA4265,Embraer,ERJ-175LR,2017,"Miami International Airport, FL (MIA/KMIA)",MIA,Miami,United States of America,"Raleigh/Durham Airport, NC (RDU/KRDU)",RDU,Raleigh/Durham,United States of America,"Miami International Airport, FL (MIA/KMIA) - U...",63,2,0,0,1,1,miscommunication,80,Miscommunication between the flight crew and t...,NTSB,completed


In [25]:
passenger.iloc[0,:].to_string()

'Date                                               Monday 8 February 2021\nTime                                                                11:02\nType                                           Embraer EMB-500 Phenom 100\nOwner/operator                                                    Luxwing\nRegistration                                                       9H-FAM\nMSN                                                              50000100\nYear of manufacture                                                2011.0\nEngine model                                Pratt & Whitney Canada PW615F\nFatalities                                   Fatalities: 0 / Occupants: 3\nOther fatalities                                                        0\nAircraft damage                                  Substantial, written off\nCategory                                                         Accident\nLocation                          Paris-Le Bourget Airport (LBG) - France\nPhase                   

In [152]:
pd.DataFrame(response.choices[0].message.parsed).set_index(0).T

,type_of_event,operating_carrier,marketing_carrier,Registration,MSN_Manufacturer_Serial_Number,flight_code,plane_manufacturer,plane_model,year_of_manufacture,from_airport,from_airport_code,from_city,from_country,to_airport,to_airport_code,to_city,to_country,incident_location,onboard_passengers,onboard_crews,fatalities,injuries_critical,injuries_minor,injuries_total,cause,cause_detailed,investigating_agency,investigation_status
1,accident,Envoy Air,American Eagle,N252NN,17000642,AA4265,Embraer,ERJ-175LR,2017,"Miami International Airport, FL (MIA/KMIA)",MIA,Miami,United States of America,"Raleigh/Durham Airport, NC (RDU/KRDU)",RDU,Raleigh/Durham,United States of America,"Miami International Airport, FL (MIA/KMIA) - U...",63,2,0,0,1,1,miscommunication,Miscommunication between the flight crew and t...,NTSB,completed


In [ ]:
Tail number?
Categorization certainty